In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
qdf = pd.read_csv("london_u6.6_5sp_5.0bmult.queue.out.csv",sep=';')

In [3]:
qdf.head(5)

,data_timestep,lane_queueing_length_experimental,lane_queueing_time,lane_id,lane_queueing_length
0,0.0,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN
2,2.0,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN
4,4.0,6.03,0.0,23028438_1,0.0


In [14]:
qdf["edge_id"] =qdf["lane_id"].str.replace("_[0-9]$","")

In [16]:
qdf[:10]

,data_timestep,lane_queueing_length_experimental,lane_queueing_time,lane_id,lane_queueing_length,edge_id
0,0.0,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN
2,2.0,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN
4,4.0,6.03,0.0,23028438_1,0.00,23028438
5,5.0,5.12,0.0,23028438_0,0.00,23028438
6,6.0,15.06,0.0,-3530411#1_0,0.00,-3530411#1
7,7.0,5.01,1.0,-3530411#1_0,5.01,-3530411#1
8,7.0,5.11,0.0,226739382#0_1,0.00,226739382#0
9,8.0,12.54,2.0,-3530411#1_0,12.54,-3530411#1


In [17]:
edge_groups = qdf.groupby("edge_id")

In [18]:
len(edge_groups)

6438

In [ ]:
if not os.path.exists(r"u6.6_5sp_5.0bmult/"):
    os.makedirs(r"u6.6_5sp_5.0bmult/")
for name, group in edge_groups:
    plt.close()
    group["lane_queueing_length"].hist(bins=10)
    #plt.savefig("u6.6_5sp_5.0bmult/" + name + "queue_length.jpg")
    group["lane_queueing_time"].hist(bins=10)
    plt.savefig("u6.6_5sp_5.0bmult/" +name + "queue_time_and_length.jpg")
    plt.close()

In [13]:
len(lane_groups)

9952

In [22]:
import scipy.stats as st
import multiprocessing

In [23]:
distributions = [st.expon, st.norm,st.gamma,st.weibull_max,st.weibull_min,st.logistic,st.beta]

In [29]:
def get_best_fits(data):
    mles = []
    for distribution in distributions:
        pars = distribution.fit(data)
        mle = distribution.nnlf(pars, data)
        mles.append(mle)
    temp = sorted(zip(distributions, mles), key=lambda d: d[1])
    return [name ,temp[0][0].name,temp[1][0].name,temp[2][0].name,temp[3][0].name,temp[4][0].name]

In [30]:
best_fits = []
for name,group in lane_groups:
    best_fits.append(get_best_fits(group["lane_queueing_length"]))

C:\Anaconda2\lib\site-packages\scipy\optimize\minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Anaconda2\lib\site-packages\scipy\optimize\minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
C:\Anaconda2\lib\site-packages\scipy\optimize\minpack.py:161: RuntimeWarning: The number of calls to function has reached maxfev = 600.
  warnings.warn(msg, RuntimeWarning)


In [31]:
best_fits

[['-100514384#1_0', 'beta', 'weibull_max', 'logistic', 'norm', 'expon'],
 ['-101323294#0_0', 'beta', 'weibull_min', 'gamma', 'expon', 'norm'],
 ['-101323294#0_1', 'weibull_min', 'gamma', 'beta', 'expon', 'logistic'],
 ['-10430399_0', 'weibull_min', 'beta', 'weibull_max', 'gamma', 'norm'],
 ['-10430399_1', 'weibull_min', 'gamma', 'beta', 'logistic', 'norm'],
 ['-10439528_0', 'beta', 'weibull_max', 'weibull_min', 'logistic', 'expon'],
 ['-104591643#0_0', 'gamma', 'beta', 'logistic', 'expon', 'weibull_min'],
 ['-10574949_0', 'weibull_min', 'gamma', 'beta', 'expon', 'norm'],
 ['-10581729#0_0', 'weibull_min', 'gamma', 'beta', 'expon', 'norm'],
 ['-10581729#1_0', 'weibull_min', 'gamma', 'beta', 'expon', 'logistic'],
 ['-108804594#1_1', 'beta', 'weibull_max', 'weibull_min', 'logistic', 'expon'],
 ['-109034688_0', 'weibull_min', 'gamma', 'beta', 'expon', 'logistic'],
 ['-109411765#0_0', 'weibull_min', 'gamma', 'beta', 'expon', 'logistic'],
 ['-109431812#4_1', 'weibull_min', 'weibull_max', 'log

In [37]:
pd.DataFrame(best_fits).to_csv("u6.6_5sp_5.0bmult/queue_best_fit.csv",index=False)